Evaluate model epoch 4 no embedding

# LSTM model

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

scaler_file = "../dataset/processed/scaler.pkl"

with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]

target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"] 

timesteps = 24
batch_size = 256

test_file = "../dataset/processed/test_data.csv"

checkpoint_path = "../model/best_model2.h5"
model = load_model(checkpoint_path)
model.summary()

def data_generator(file_path, feature_cols, target_cols, batch_size=256, timesteps=24):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):
        chunk = chunk.sort_values(by=["Datetime"])
        
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])

        yield np.array(X_batch), np.array(y_batch)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_file, feature_cols, target_cols, batch_size, timesteps),
    output_signature=(
        tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32),
    )
).prefetch(tf.data.AUTOTUNE)

loss, mae = model.evaluate(test_dataset)
print(f"✅ Kết quả đánh giá trên dữ liệu test: Loss={loss:.4f}, MAE={mae:.4f}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           72192     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 144)               9360      
                                                                 
 reshape (Reshape)           (None, 24, 6)             0         
                                                                 
Total params: 130,960
Trainable params: 130,960
Non-trai

# Fine-tune & CNN-LSTM

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

scaler_file = "../dataset/processed/scaler.pkl"
with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]
target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"] 

timesteps = 24
batch_size = 256

test_file = "../dataset/processed/test_data.csv"
checkpoint_path = "best_model_spatial.h5"
model = load_model(checkpoint_path)

print("Model: \"sequential\"")
model.summary()

def data_generator(file_path, feature_cols, target_cols, batch_size=256, timesteps=24):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):
        chunk = chunk.sort_values(by=["Datetime", "Latitude", "Longitude"])
        
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])

        yield np.array(X_batch), np.array(y_batch)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_file, feature_cols, target_cols, batch_size, timesteps),
    output_signature=(
        tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32),
    )
).prefetch(tf.data.AUTOTUNE)

print("\nĐang đánh giá mô hình trên tập test...")
loss, mae = model.evaluate(test_dataset, verbose=1)
print(f"✅ Kết quả đánh giá trên dữ liệu test: Loss={loss:.4f}, MAE={mae:.4f}")

Model: "sequential"
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           72192     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 144)               9360      
                                                                 
 reshape (Reshape)           (None, 24, 6)             0         
                                                                 
Total params: 130,960
Trainable para

# Fine-tune LSTM

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

scaler_file = "../dataset/processed/scaler.pkl"
with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]
target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"] 

timesteps = 24
batch_size = 256

test_file = "../dataset/processed/test_data.csv"
checkpoint_path = "best_model_spatial.h5"
model = load_model(checkpoint_path)

print("Model: \"sequential\"")
model.summary()

def data_generator(file_path, feature_cols, target_cols, batch_size=256, timesteps=24):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):
        chunk = chunk.sort_values(by=["Datetime", "Latitude", "Longitude"])
        
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])

        yield np.array(X_batch), np.array(y_batch)

test_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(test_file, feature_cols, target_cols, batch_size, timesteps),
    output_signature=(
        tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32), 
        tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32),
    )
).prefetch(tf.data.AUTOTUNE)

print("\nĐang đánh giá mô hình trên tập test...")
loss, mae = model.evaluate(test_dataset, verbose=1)
print(f"✅ Kết quả đánh giá trên dữ liệu test: Loss={loss:.4f}, MAE={mae:.4f}")

Model: "sequential"
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           72192     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 144)               9360      
                                                                 
 reshape (Reshape)           (None, 24, 6)             0         
                                                                 
Total params: 130,960
Trainable para

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import gc

# Load scaler
scaler_file = "../dataset/processed/scaler.pkl"
with open(scaler_file, 'rb') as f:
    scaler_X, scaler_y = pickle.load(f)

# Định nghĩa các cột đầu vào và đầu ra
feature_cols = ["Latitude", "Longitude", "hour", "day", "month", "season", 
                "WS10M", "QV2M", "PS", "PRECTOTCORR", "T2M", "CLRSKY_SFC_SW_DWN"]
target_cols = ["CLRSKY_SFC_SW_DWN", "PS", "T2M", "QV2M", "WS10M", "PRECTOTCORR"] 

timesteps = 24
batch_size = 64  # Giảm kích thước batch để tránh tràn bộ nhớ

# Tải mô hình đã huấn luyện trước
checkpoint_path = "best_model_spatial.h5"  # Đảm bảo đường dẫn đúng
model = load_model(checkpoint_path)

# Kiểm tra mô hình
print("Model: \"sequential\"")
model.summary()

# Định nghĩa data generator để chuẩn bị dữ liệu cho mô hình
def data_generator(file_path, feature_cols, target_cols, batch_size=64, timesteps=24):
    dtype_dict = {col: np.float32 for col in feature_cols + target_cols}
    dtype_dict.update({"hour": np.int8, "day": np.int8, "month": np.int8, "season": np.int8})

    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):
        chunk = chunk.sort_values(by=["Datetime", "Latitude", "Longitude"])
        
        # Chuẩn hóa dữ liệu
        X_scaled = scaler_X.transform(chunk[feature_cols])
        y_scaled = scaler_y.transform(chunk[target_cols])

        X_batch, y_batch = [], []
        for i in range(len(chunk) - timesteps - 24):
            X_batch.append(X_scaled[i:i+timesteps])  # Lưu batch dữ liệu đầu vào
            y_batch.append(y_scaled[i+timesteps:i+timesteps+24])  # Lưu batch dữ liệu đầu ra

        # Trả về batch dưới dạng np.array với đúng shape
        yield np.array(X_batch), np.array(y_batch)

# Đường dẫn đến file dữ liệu test
test_file = "../dataset/processed/test_data.csv"

# Đọc tệp dữ liệu theo chunk và lấy 20% cuối từ mỗi chunk
chunk_size = 10000  # Bạn có thể điều chỉnh kích thước của chunk
test_data_chunks = pd.read_csv(test_file, chunksize=chunk_size)

# Khởi tạo các biến để lưu trữ kết quả
y_true = []
y_pred = []
errors = []

# Lặp qua từng chunk để xử lý và đánh giá mô hình
for chunk in test_data_chunks:
    # Lấy 20% cuối của mỗi chunk
    test_subset = chunk.tail(int(len(chunk) * 0.2))  # Lấy 20% cuối
    
    # Tạo dataset kiểm tra từ phần dữ liệu con (subset)
    test_dataset_subset = tf.data.Dataset.from_generator(
        lambda: data_generator(test_subset, feature_cols, target_cols, batch_size=batch_size, timesteps=24),
        output_signature=(tf.TensorSpec(shape=(None, timesteps, len(feature_cols)), dtype=tf.float32), 
                          tf.TensorSpec(shape=(None, 24, len(target_cols)), dtype=tf.float32))
    ).prefetch(tf.data.AUTOTUNE)

    # Đánh giá mô hình trên tập test con (subset)
    loss, mae = model.evaluate(test_dataset_subset, verbose=1)
    print(f"✅ Kết quả đánh giá trên dữ liệu test con (chunk): Loss={loss:.4f}, MAE={mae:.4f}")

    # Tính lỗi cho từng mẫu trong tập kiểm tra con
    for X_batch, y_batch in test_dataset_subset:
        y_true.append(y_batch)
        y_pred.append(model.predict(X_batch))

# Chuyển danh sách thành mảng numpy
y_true = np.concatenate(y_true, axis=0)
y_pred = np.concatenate(y_pred, axis=0)

# Tính MAE (Lỗi tuyệt đối trung bình) cho toàn bộ tập kiểm tra
mae = mean_absolute_error(y_true, y_pred)
print(f"MAE (Toàn bộ tập kiểm tra con): {mae}")

# Tính MSE và RMSE
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
print(f"MSE: {mse}, RMSE: {rmse}")

# Tính R² (R-squared)
r2 = r2_score(y_true, y_pred)
print(f"R²: {r2}")

# Phân tích lỗi: Tính lỗi tuyệt đối cho mỗi mẫu
errors = np.abs(y_true - y_pred)

# Vẽ histogram phân phối lỗi để kiểm tra sự phân bố
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50, color='blue', edgecolor='black')
plt.title("Phân phối lỗi (MAE)")
plt.xlabel("Lỗi tuyệt đối")
plt.ylabel("Tần suất")
plt.show()

# Phân tích các điểm dữ liệu có lỗi lớn (các điểm có lỗi cao)
error_threshold = 0.1  # Bạn có thể điều chỉnh ngưỡng này tùy theo dữ liệu của bạn
high_error_indices = np.where(errors > error_threshold)[0]  # Các chỉ số có lỗi lớn

# In ra những chỉ số lỗi lớn
print(f"Số lượng mẫu có lỗi lớn: {len(high_error_indices)}")
print(f"Chỉ số các mẫu có lỗi lớn: {high_error_indices[:10]}")  # In 10 mẫu đầu tiên có lỗi lớn

# Hiển thị các mẫu có lỗi lớn
high_error_data = pd.DataFrame({
    'y_true': y_true[high_error_indices],
    'y_pred': y_pred[high_error_indices],
    'error': errors[high_error_indices]
})

# In ra các lỗi lớn để kiểm tra
print(high_error_data.head())

# Kiểm tra lỗi theo tọa độ
latitude = np.concatenate([X_batch[:, :, 0] for X_batch, _ in test_dataset_subset], axis=0)  # Giả sử latitudes là cột đầu tiên trong X
longitude = np.concatenate([X_batch[:, :, 1] for X_batch, _ in test_dataset_subset], axis=0)  # Giả sử longitudes là cột thứ hai trong X

# Tạo DataFrame chứa tọa độ và lỗi
df = pd.DataFrame({
    'Latitude': latitude,
    'Longitude': longitude,
    'y_true': y_true.flatten(),
    'y_pred': y_pred.flatten(),
    'error': errors.flatten()
})

# Tính MAE theo tọa độ (tính MAE cho mỗi cặp latitude, longitude)
mae_by_location = df.groupby(['Latitude', 'Longitude'])['error'].mean()

# Chuyển kết quả thành DataFrame và reset index để sử dụng pivot
mae_by_location = mae_by_location.reset_index()

# Vẽ heatmap MAE theo tọa độ
mae_by_location_pivot = mae_by_location.pivot(index='Latitude', columns='Longitude', values='error')
plt.figure(figsize=(12, 8))
sns.heatmap(mae_by_location_pivot, cmap="coolwarm", annot=True, fmt=".2f")
plt.title("Heatmap MAE theo tọa độ")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

# Kiểm tra các khu vực có MAE cao hơn
high_mae_locations = mae_by_location[mae_by_location['error'] > error_threshold]
print("Các tọa độ có MAE cao:")
print(high_mae_locations)


Model: "sequential"
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 128)           72192     
                                                                 
 dropout (Dropout)           (None, 24, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 144)               9360      
                                                                 
 reshape (Reshape)           (None, 24, 6)             0         
                                                                 
Total params: 130,960
Trainable para

InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  TypeError: argument of type 'method' is not iterable
Traceback (most recent call last):

  File "d:\anaconda3\envs\weather_lstm\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "d:\anaconda3\envs\weather_lstm\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "d:\anaconda3\envs\weather_lstm\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\tranh\AppData\Local\Temp\ipykernel_21592\715416960.py", line 37, in data_generator
    for chunk in pd.read_csv(file_path, chunksize=batch_size * 5, dtype=dtype_dict, parse_dates=["Datetime"], low_memory=True):

  File "d:\anaconda3\envs\weather_lstm\lib\site-packages\pandas\io\parsers\readers.py", line 1026, 